In [170]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [171]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present.
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)

In [172]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    # df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

Lặp đơn

In [173]:
def check_norm (A: np.ndarray):
    tmp = np.abs(A);
    print("Norm column: ", np.linalg.norm(tmp, ord=1))
    print("Norm 2: ", np.linalg.norm(tmp, ord=2))
    print("Norm row: ", np.linalg.norm(tmp, ord=np.inf))
    print("Norm max: ", 3*np.max(tmp))

# Thuật toán

In [174]:
def fixed_point_matrix_iteration(A, B, initial_values, q, eps, eta, norm):
    """
    Perform fixed-point iteration for system of linear equations using x_new = Ax + B
    
    Parameters:
    A (numpy.ndarray): Coefficient matrix
    B (numpy.ndarray): Constant vector
    initial_values (numpy.ndarray): Initial guess for x
    q (float): Contraction coefficient
    eps (float): Tolerance for convergence
    norm (int): Type of norm to use (1 for L1 norm, -1 for max norm)
    
    Returns:
    numpy.ndarray: Solution vector
    """
    if (eps is None) == (eta is None):
        raise ValueError("Specify exactly one of eps (exact) or eta (relative)")
    
    if norm == 1:
        vec_norm = lambda x: np.sum(np.abs(x))
    elif norm == -1:
        vec_norm = lambda x: np.max(np.abs(x))
    
    n = len(initial_values)
    values = np.array(initial_values)
    results = [[0] + initial_values.tolist()]

    #Calculate the shrinking speed q:
    new_eps = (eps if eps is not None else eta) * (1-q) / q
    print(f"new_eps: {new_eps:.12f}")

    while True:
        # Calculate new values using matrix operation x_new = Ax + B
        new_values = np.dot(A, values) + B
        
        # Calculate the differences
        if eps is not None:
            total_diff = vec_norm(new_values - values)
        else:
            total_diff = vec_norm(new_values - values) / vec_norm(new_values)
        
        # Append results
        results.append(new_values.tolist() + [total_diff])
        values = new_values
        
        # Check for convergence
        if total_diff < new_eps:
            break
        
    columns = [f'x{j+1}' for j in range(n)] + ['total_diff']
    df = pd.DataFrame(results, columns=columns)
    df.index.name = "Iteration"
    return df

# Kết quả

In [175]:
#Original matrix Ax=B
A = input_matrix('FXP_input_A.txt', convert_fractions=True)
B = input_matrix('FXP_input_B.txt', convert_fractions=True).flatten() #remove flatten if B is multi-column matrix

print("\nMatrix A:"); output_matrix(A)
check_norm(A)
print("\nMatrix B:"); output_matrix(B)


Matrix A:
0.1588 0.0064 0.0025 0.0304 0.0014 0.0083 0.1594
0.0057 0.2645 0.0436 0.0099 0.0083 0.0201 0.3413
0.0264 0.1506 0.3557 0.0139 0.0142 0.0070 0.0236
0.3299 0.0565 0.0495 0.3636 0.0204 0.0483 0.0649
0.0089 0.0081 0.0333 0.0295 0.3412 0.0237 0.0020
0.1190 0.0901 0.0996 0.1260 0.1722 0.2368 0.3369
0.0063 0.0126 0.0196 0.0098 0.0064 0.0132 0.0012
Norm column:  0.9292999999999998
Norm 2:  0.7286368954281354
Norm row:  1.1806
Norm max:  1.0908

Matrix B:
 74000.0
 56000.0
 10500.0
 25000.0
 17500.0
196000.0
  5000.0


In [176]:
# Initial guess (zero vector or any other initial guess)
initial_guess = np.array([0, 0, 0, 0, 0, 0, 0])
q = np.linalg.norm(np.abs(A), ord=1)
eps = 1e-6
eta = None
norm = 1

# Call the function
df_history = fixed_point_matrix_iteration (A, B, initial_guess, q, eps, eta, norm)
print(df_history)

new_eps: 0.000000076079
                           x1                  x2                  x3                   x4                  x5                   x6                  x7           total_diff
Iteration                                                                                                                                                                   
0              0.000000000000      0.000000000000      0.000000000000       0.000000000000      0.000000000000       0.000000000000      0.000000000000       0.000000000000
1          74000.000000000000  56000.000000000000  10500.000000000000   25000.000000000000  17500.000000000000  196000.000000000000   5000.000000000000  384000.000000000000
2          89344.149999999994  77730.449999999997  26708.050000000003   72334.649999999994  30325.550000000003  265158.200000000012   9327.799999999999  186928.849999999977
3          94681.189534999998  87714.509720000002  37577.262640000001  100520.489224999998  38598.012419999999 

In [178]:
solution_series = df_history.filter(regex=r'^x\d+$').iloc[-1]
print("Approximate solution:"),
print(solution_series.to_string())

Approximate solution:
x1     99575.653397644855
x2     97703.022863484832
x3     51230.523166375737
x4    131569.921928706754
x5     49488.491372352932
x6    329554.452569977555
x7     13835.335715011599
